# Computer Vision - Semester 8, Spring 2021

## Lab Project 2: Optical Flow Estimation and Video Feature Extraction for Action Recognition
    
    Christos Dimopoulos (031 17 037)
    Dimitris Dimos (031 17 165)

### Part 2: Spacio-Temporal Interest Points Detection and Feature Extraction in Human Action Videos 

In [50]:
# imports
from cv21_lab2_2_utils import read_video, show_detection, orientation_histogram, bag_of_words, svm_train_test
from os import listdir
from os.path import isfile, join
import numpy as np
import scipy
from scipy import signal
from scipy import ndimage as nd
import cv2

In [51]:
# typical values for our tests
typ = {
    'sigma': 4,
    'tau': 1.5,
    'k': 0.005,
    's': 2
}

In [59]:
# input videos

walking_videos = ['./data/walking/'+f for f in listdir('./data/walking/') if isfile(join('./data/walking/', f))]
running_videos = ['./data/running/'+f for f in listdir('./data/running/') if isfile(join('./data/running/', f))]
boxing_videos  = ['./data/boxing/' +f for f in listdir('./data/boxing/')  if isfile(join('./data/boxing/' , f))]
boxing_videos.remove('./data/boxing/.DS_Store')

walk_video1 = read_video(walking_videos[1], 200)
run_video1  = read_video(running_videos[1], 200)
box_video1  = read_video(boxing_videos[1],  200) 

walk_video2 = read_video(walking_videos[4], 200)
run_video2  = read_video(running_videos[4], 200)
box_video2  = read_video(boxing_videos[4],  200) 

walk_video3 = read_video(walking_videos[8], 200)
run_video3  = read_video(running_videos[8], 200)
box_video3  = read_video(boxing_videos[8],  200) 

#### 2.1 Spatio-Temporal Points of Interest

In [53]:
# auxiliary functions

def gaussian_1D (sigma):
    kernelSize = int (np.ceil(3*sigma)*2 + 1)
    G = cv2.getGaussianKernel(kernelSize, sigma)
    return G

def gaussian_2D (sigma):
    G_1D = gaussian_1D(sigma)
    G_2D = G_1D @ G_1D.transpose()
    return G_2D

''' Gets the importance criterion H and returns a (N,4) matrix containing points of interest '''
def calculate_interest_points(H, N, sigma):
    
    N_largest = np.dstack(np.unravel_index(np.argsort(H.ravel()), (H.shape[0], H.shape[1], H.shape[2])))
    N_largest = N_largest.reshape(N_largest.shape[1], N_largest.shape[2])
    N_largest = N_largest[-N:]
    
    toReturn = np.array([np.append(pair[:2][::-1],[pair[2], sigma]) for pair in N_largest])
        
    return toReturn

In [54]:
# 2.1.1 - Harris Detector

'''
Usage: criterion = HarrisDetector(V, sigma, tau, k, s)

Description:
    Implements Harris interest points detector and returns
    the importance criterion for the input video
 
Input:
    V: input video
    sigma: initial spatial scale
    tau: initial temporal scale
    k: threshold
    s: scaling parameter
   
Output:
    H: Harris importance criterion
'''
def HarrisDetector (V, sigma, tau, k, s):
    
    # convert voxels to floats
    V = V.astype(np.float32)
    
    # create a spatial Gaussian kernel
    Gspace = gaussian_2D(sigma)
    Gspace = Gspace.reshape(Gspace.shape[0], Gspace.shape[1], 1) # !!!!
    
    # create a temporal Gaussian kernel
    Gtime = gaussian_1D(tau)
    Gtime = Gtime.reshape(1,1,Gtime.shape[0])
    
    # spatio-temporal filtering of the input video
    G_spatio_temporal = nd.convolve(Gtime, Gspace)
    L = nd.convolve(V, G_spatio_temporal)
    
    # derivative filters
    dx_filter = np.array([[[-1],[0],[1]]], dtype=np.int8) # !!!!
    dy_filter = np.array([[[-1]],[[0]],[[1]]], dtype=np.int8) # !!!!
    dt_filter = np.array([[[-1,  0,  1]]], dtype=np.int8)
    
    # calculate L derivatives
    Lx = nd.convolve(L, dx_filter)
    Ly = nd.convolve(L, dy_filter)
    Lt = nd.convolve(L, dt_filter)
    
    # calculate elements of M BEFORE convolution with g(x, y, t; s*sigma, s*tau)
    Lxx = Lx*Lx
    Lyy = Ly*Ly
    Ltt = Lt*Lt
    Lxy = Lx*Ly
    Lxt = Lx*Lt
    Lyt = Ly*Lt
    
    # calculate smoothing kernel g(x, y, t; s*sigma, s*tau)
    g_space = gaussian_2D(s*sigma)
    g_space = g_space.reshape(g_space.shape[0], g_space.shape[1], 1) # !!!!
    
    g_time = gaussian_1D(s*tau)
    g_time = g_time.reshape(1, 1, g_time.shape[0])
    
    g_spatio_temporal = nd.convolve(g_time, g_space)
    
    # calculate M
    Mxx = nd.convolve(Lxx, g_spatio_temporal)
    Myy = nd.convolve(Lyy, g_spatio_temporal)
    Mtt = nd.convolve(Ltt, g_spatio_temporal)
    Mxy = nd.convolve(Lxy, g_spatio_temporal)
    Mxt = nd.convolve(Lxt, g_spatio_temporal)
    Myt = nd.convolve(Lyt, g_spatio_temporal)
    
    # calculate cornerness criterion
    trace = Mxx + Myy + Mtt
    det = (Mxx*Myy*Mtt - Mxx*Myt**2) - (Mtt*Mxy**2 - Mxy*Mxt*Myt) + (Mxy*Mxt*Myt - Myy*Mxt**2)
    H = det - k*trace**3
    
    return H

In [55]:
# 2.1.2 - Gabor Detector
'''
Usage: criterion = GaborDetector(V, sigma, tau, k, s)

Description:
    Implements Gabor interest points detector and returns
    the importance criterion for the input video
 
Input:
    V: input video
    sigma: initial spatial scale
    tau: initial temporal scale
   
Output:
    H: Gabor importance criterion
'''

def GaborDetector (V, sigma, tau):
    
    # convert voxels to floats
    V = V.astype(np.float32)
    
    # spatial smoothing with gaussian kernel
    Gspace = gaussian_2D(sigma)
    Gspace = Gspace.reshape(Gspace.shape[0], Gspace.shape[1], 1) # !!!
    Vs = nd.convolve(V, Gspace)
    
    # construct gabor filters
    time = np.linspace(int(-2*tau), int(2*tau), int(4*tau+1), endpoint=True)
    w = 4/tau
    h_ev = -np.cos(2*np.pi*time*w)*np.exp((-time**2)/(2*tau**2))
    h_ev = h_ev/np.linalg.norm(h_ev, ord=1)
    h_od = -np.sin(2*np.pi*time*w)*np.exp((-time**2)/(2*tau**2))
    h_od = h_od/np.linalg.norm(h_od, ord=1)
    
    # reshaping filters to convolve in the right dimension
    h_ev = h_ev.reshape(1, 1, h_ev.shape[0])
    h_od = h_od.reshape(1, 1, h_od.shape[0])
    
    # calculating importance criterion
    H = (nd.convolve(Vs, h_ev))**2 + (nd.convolve(Vs, h_od))**2
    
    return H

In [58]:
# 2.1.3 - Experiments

# Harris Detector

# video 1
walk_harris_criterion = HarrisDetector(walk_video3, 4, 1.5, 0.005, 2)
run_harris_criterion  = HarrisDetector(run_video3,  4, 1.5, 0.005, 2)
box_harris_criterion  = HarrisDetector(box_video3,  4, 1.5, 0.005, 2)

walk_harris_points = calculate_interest_points(walk_harris_criterion, 600, 4)
run_harris_points  = calculate_interest_points(run_harris_criterion,  600, 4)
box_harris_points  = calculate_interest_points(box_harris_criterion,  600, 4)

show_detection(walk_video3, walk_harris_points, save_path=r"./HarrisDetection/walk")
show_detection(run_video3,  run_harris_points,  save_path=r"./HarrisDetection/run")
show_detection(box_video3,  box_harris_points,  save_path=r"./HarrisDetection/box")

!ffmpeg -i ./HarrisDetection/walk/frame%d.png -c:v libx264 -vf fps=25 ./HarrisDetection/harris_walk.mp4
!ffmpeg -i ./HarrisDetection/run/frame%d.png -c:v libx264 -vf fps=25 ./HarrisDetection/harris_run.mp4
!ffmpeg -i ./HarrisDetection/box/frame%d.png -c:v libx264 -vf fps=25 ./HarrisDetection/harris_box.mp4

ffmpeg version 4.4 Copyright (c) 2000-2021 the FFmpeg developers
  built with Apple clang version 12.0.0 (clang-1200.0.32.29)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/4.4_1 --enable-shared --enable-pthreads --enable-version3 --enable-avresample --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --disable-indev=jack --enable-videotoolbox
  libavutil      

ffmpeg version 4.4 Copyright (c) 2000-2021 the FFmpeg developers
  built with Apple clang version 12.0.0 (clang-1200.0.32.29)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/4.4_1 --enable-shared --enable-pthreads --enable-version3 --enable-avresample --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --disable-indev=jack --enable-videotoolbox
  libavutil      

In [57]:
# Gabor Detector

# video 1
walk_gabor_criterion = GaborDetector(walk_video3, 1.6, 1.5)
run_gabor_criterion  = GaborDetector(run_video2,  1.6, 1.5)
box_gabor_criterion  = GaborDetector(box_video1,  1.6, 1.5)

walk_gabor_points = calculate_interest_points(walk_gabor_criterion, 600, 1.6)
run_gabor_points  = calculate_interest_points(run_gabor_criterion,  600, 1.6)
box_gabor_points  = calculate_interest_points(box_gabor_criterion,  600, 1.6)

show_detection(walk_video3, walk_gabor_points, save_path=r"./GaborDetection/walk")
show_detection(run_video2,  run_gabor_points,  save_path=r"./GaborDetection/run")
show_detection(box_video1,  box_gabor_points,  save_path=r"./GaborDetection/box")

!ffmpeg -i ./GaborDetection/walk/frame%d.png -c:v libx264 -vf fps=25 ./GaborDetection/gabor_walk.mp4
!ffmpeg -i ./GaborDetection/run/frame%d.png -c:v libx264 -vf fps=25 ./GaborDetection/gabor_run.mp4
!ffmpeg -i ./GaborDetection/box/frame%d.png -c:v libx264 -vf fps=25 ./GaborDetection/gabor_box.mp4

ffmpeg version 4.4 Copyright (c) 2000-2021 the FFmpeg developers
  built with Apple clang version 12.0.0 (clang-1200.0.32.29)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/4.4_1 --enable-shared --enable-pthreads --enable-version3 --enable-avresample --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --disable-indev=jack --enable-videotoolbox
  libavutil      

ffmpeg version 4.4 Copyright (c) 2000-2021 the FFmpeg developers
  built with Apple clang version 12.0.0 (clang-1200.0.32.29)
  configuration: --prefix=/usr/local/Cellar/ffmpeg/4.4_1 --enable-shared --enable-pthreads --enable-version3 --enable-avresample --cc=clang --host-cflags= --host-ldflags= --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libbluray --enable-libdav1d --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencore-amrwb --enable-libopenjpeg --enable-libspeex --enable-libsoxr --enable-libzmq --enable-libzimg --disable-libjack --disable-indev=jack --enable-videotoolbox
  libavutil      

#### 2.2 Spatio-Temporal Histographic Descriptors

In [39]:
# 2.1.1 - Calculating Gradient and Optical Flow

def get_gradient(V):
    dy, dx, _ = np.gradient(V)
    return (dy,dx)

def get_optical_flow(V):
    
    V = V.astype(np.uint8)
    
    flow_x = np.zeros((V.shape[0], V.shape[1], V.shape[2]))
    flow_y = np.zeros((V.shape[0], V.shape[1], V.shape[2]))
    for f in range(V.shape[2]):
        t = f
        if f == V.shape[2]-1:
            t = f-1
                
        temp = cv2.optflow.DualTVL1OpticalFlow_create(nscales=1).calc(V[:,:,t], V[:,:,t+1], None)
        flow_x[:,:,t] = temp[:,:,1]
        flow_y[:,:,t] = temp[:,:,0]
        
    print(u'\u2713', end='')
    return (flow_y,flow_x)
    

In [40]:
# 2.2.2 - HOG and HOF descriptors

def get_descriptor (V, points, descriptor, nbins=9, n=3, m=3):
    
    height = V.shape[0]
    width  = V.shape[1]
    desc = []
    
    if descriptor == 'HOG':
        dy, dx = get_gradient(V)
    elif descriptor == 'HOF':
        dy, dx = get_optical_flow(V)
    elif descriptor == 'HOG/HOF':
        HOG = get_descriptor(V, points, 'HOG', nbins, n, m)
        HOF = get_descriptor(V, points, 'HOF', nbins, n, m)
        return np.concatenate((HOG, HOF))
    else:
        print('Invalid Descriptor Type')
        return
    
    for point in points:
        x = point[0]
        y = point[1]
        t = point[2]
        sigma = point[3]
        side = int (np.round(4*sigma))
        
        x_left  = max(0, x-side)
        x_right = min(width, x+side+1)
        y_top   = max(0, y-side)
        y_bot   = min(height, y+side+1) 
        
        Gx = dx[y_top:y_bot, x_left:x_right, t]
        Gy = dy[y_top:y_bot, x_left:x_right, t]
        
        desc.append( orientation_histogram(Gx, Gy, nbins, np.array([n,m])) )
    
    return np.array(desc)

#### 2.3 Bag of Visual Words Construction and use of Support Vector Machines for action classification

In [41]:
# 2.3.1 - Dividing data in train and test set

def divide_into_train_test(training_videos_file):

    temp_file = open('./data/'+training_videos_file, 'r')
    lines = temp_file.readlines()
    train_names = [i.strip() for i in lines]

    train_set_temp = []
    test_set_temp = []

    train_labels = []
    test_labels = []

    for name in listdir('./data/walking/'):
        if name in train_names:
            train_set_temp.append(read_video('./data/walking/'+name, 200))
            train_labels.append(2)
        else:
            test_set_temp.append(read_video('./data/walking/'+name, 200))
            test_labels.append(2)

    for name in listdir('./data/running/'):
        if name in train_names:
            train_set_temp.append(read_video('./data/running/'+name, 200))
            train_labels.append(0)
        else:
            test_set_temp.append(read_video('./data/running/'+name, 200))
            test_labels.append(0)

    for name in listdir('./data/boxing/'):
        if name in train_names:
            train_set_temp.append(read_video('./data/boxing/'+name, 200))
            train_labels.append(1)
        elif name != '.DS_Store':
            test_set_temp.append(read_video('./data/boxing/'+name, 200))
            test_labels.append(1)

    train_set = np.array(train_set_temp)
    test_set = np.array(test_set_temp)
    
    return train_set, test_set, train_labels, test_labels

In [42]:
# 2.3.2 - Creating Bag of Visual Words Representation

def calculate_descriptor_for_video_set(descriptor, detector, video_set, sigma, tau, k, s):
    
    n = int(np.round(4*sigma))
    m = int(np.round(4*sigma))
    nbins = 9
    
    if descriptor != 'HOG' and descriptor != 'HOF' and descriptor != 'HOG/HOF':
        print('Invalid Descriptor')
        return
    
    descriptors_for_video_set = []
    
    if detector == 'Harris':
        for video in video_set:
            criterion = HarrisDetector(video, sigma, tau, k, s)
            points = calculate_interest_points(criterion, 600, sigma)
            desc = get_descriptor(video, points, descriptor, nbins, n, m)
            descriptors_for_video_set.append(desc)
            
    elif detector == 'Gabor':
        for video in video_set:
            criterion = GaborDetector(video, sigma, tau)
            points = calculate_interest_points(criterion, 600, sigma)
            desc = get_descriptor(video, points, descriptor, nbins, n, m)
            descriptors_for_video_set.append(desc)
            
    else:
        print('Invalid Detector')
        return
    
    return descriptors_for_video_set


def calculate_BoVW(train_set, test_set, descriptor, detector, sigma, tau, k, s, num_centers):
    
    desc_train = calculate_descriptor_for_video_set(descriptor, detector, train_set, sigma, tau, k, s)
    desc_test  = calculate_descriptor_for_video_set(descriptor, detector, test_set,  sigma, tau, k, s)
    
    bow_train, bow_test = bag_of_words(desc_train, desc_test, num_centers)
    
    return (bow_train, bow_test)

In [43]:
# 2.3.3 - Classification using Support Vector Machine

def classify (train_set, test_set, train_labels, test_labels, descriptor, detector, sigma, tau, k, s, num_centers=20):
    
    print("Performing Classification using the following parameters:")
    print("Detector: {},".format(detector), end=' ')
    print("Descriptor: {},".format(descriptor), end=' ')
    print("sigma = {0:.2f}, tau = {1:.2f}".format(sigma, tau))
    print()
    
    bow_train, bow_test = calculate_BoVW(train_set, test_set, descriptor, detector, sigma, tau, k, s, num_centers)
    accuracy, pred = svm_train_test(bow_train, train_labels, bow_test, test_labels)
    
    print("\nClassification finished. The results are:")
    print("\t Accuracy = {0:.2f}%".format(accuracy*100))
    print("\t Pred = ", end="")
    print(pred)
    
    return

In [45]:
# 2.3.4 - Experiments with different detector-descriptor combinations

train_set, test_set, train_labels, test_labels = divide_into_train_test('training_videos.txt')
sigma = typ['sigma']
tau = typ['tau']
k = typ['k']
s = typ['s']
num_centers = 20

# Detector: Harris,  Descriptor: HOG
print('----------------\nClassification 1\n----------------')
classify(train_set, test_set, train_labels, test_labels, 'HOG', 'Harris', sigma, tau, k, s, 20)
print()

# Detector: Gabor,   Descriptor: HOG
print('----------------\nClassification 2\n----------------')
classify(train_set, test_set, train_labels, test_labels, 'HOG', 'Gabor', sigma, tau, k, s, 20)
print()

# Detector: Harris,  Descriptor: HOF
print('----------------\nClassification 3\n----------------')
classify(train_set, test_set, train_labels, test_labels, 'HOF', 'Harris', sigma, tau, k, s, 20)
print()

# Detector: Gabor,   Descriptor: HOF
print('----------------\nClassification 4\n----------------')
classify(train_set, test_set, train_labels, test_labels, 'HOF', 'Gabor', sigma, tau, k, s, 20)
print()

# Detector: Harris,  Descriptor: HOG/HOF
print('----------------\nClassification 5\n----------------')
classify(train_set, test_set, train_labels, test_labels, 'HOG/HOF', 'Harris', sigma, tau, k, s, 20)
print()

# Detector: Gabor,   Descriptor: HOG/HOF
print('----------------\nClassification 6\n----------------')
classify(train_set, test_set, train_labels, test_labels, 'HOG/HOF', 'Gabor', sigma, tau, k, s, 20)
print()

----------------
Classification 1
----------------
Performing Classification using the following parameters:
Detector: Harris, Descriptor: HOG, sigma = 4.00, tau = 1.50


Classification finished. The results are:
	 Accuracy = 91.67%
	 Pred = [2 2 2 2 0 1 0 0 1 1 1 1]

----------------
Classification 2
----------------
Performing Classification using the following parameters:
Detector: Gabor, Descriptor: HOG, sigma = 4.00, tau = 1.50


Classification finished. The results are:
	 Accuracy = 91.67%
	 Pred = [2 2 2 2 0 1 0 0 1 1 1 1]

----------------
Classification 3
----------------
Performing Classification using the following parameters:
Detector: Harris, Descriptor: HOF, sigma = 4.00, tau = 1.50

✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓
Classification finished. The results are:
	 Accuracy = 83.33%
	 Pred = [2 0 2 2 0 0 1 0 1 1 1 1]

----------------
Classification 4
----------------
Performing Classification using the following parameters:
Detector: Gabor, Descriptor: HOF, sig

In [46]:
# Re-running the same experiments to observe the randomness

train_set, test_set, train_labels, test_labels = divide_into_train_test('training_videos.txt')
sigma = typ['sigma']
tau = typ['tau']
k = typ['k']
s = typ['s']
num_centers = 20

# Detector: Harris,  Descriptor: HOG
print('----------------\nClassification 1\n----------------')
classify(train_set, test_set, train_labels, test_labels, 'HOG', 'Harris', sigma, tau, k, s, 20)
print()

# Detector: Gabor,   Descriptor: HOG
print('----------------\nClassification 2\n----------------')
classify(train_set, test_set, train_labels, test_labels, 'HOG', 'Gabor', sigma, tau, k, s, 20)
print()

# Detector: Harris,  Descriptor: HOF
print('----------------\nClassification 3\n----------------')
classify(train_set, test_set, train_labels, test_labels, 'HOF', 'Harris', sigma, tau, k, s, 20)
print()

# Detector: Gabor,   Descriptor: HOF
print('----------------\nClassification 4\n----------------')
classify(train_set, test_set, train_labels, test_labels, 'HOF', 'Gabor', sigma, tau, k, s, 20)
print()

# Detector: Harris,  Descriptor: HOG/HOF
print('----------------\nClassification 5\n----------------')
classify(train_set, test_set, train_labels, test_labels, 'HOG/HOF', 'Harris', sigma, tau, k, s, 20)
print()

# Detector: Gabor,   Descriptor: HOG/HOF
print('----------------\nClassification 6\n----------------')
classify(train_set, test_set, train_labels, test_labels, 'HOG/HOF', 'Gabor', sigma, tau, k, s, 20)
print()

----------------
Classification 1
----------------
Performing Classification using the following parameters:
Detector: Harris, Descriptor: HOG, sigma = 4.00, tau = 1.50


Classification finished. The results are:
	 Accuracy = 91.67%
	 Pred = [2 2 2 2 0 1 0 0 1 1 1 1]

----------------
Classification 2
----------------
Performing Classification using the following parameters:
Detector: Gabor, Descriptor: HOG, sigma = 4.00, tau = 1.50


Classification finished. The results are:
	 Accuracy = 100.00%
	 Pred = [2 2 2 2 0 0 0 0 1 1 1 1]

----------------
Classification 3
----------------
Performing Classification using the following parameters:
Detector: Harris, Descriptor: HOF, sigma = 4.00, tau = 1.50

✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓
Classification finished. The results are:
	 Accuracy = 91.67%
	 Pred = [2 2 2 2 0 0 1 0 1 1 1 1]

----------------
Classification 4
----------------
Performing Classification using the following parameters:
Detector: Gabor, Descriptor: HOF, si

In [47]:
# 2.3.5 - Experiments with different data partitions

# Very little data

train_set, test_set, train_labels, test_labels = divide_into_train_test('little_train.txt')
sigma = typ['sigma']
tau = typ['tau']
k = typ['k']
s = typ['s']
num_centers = 20

# Detector: Gabor,   Descriptor: HOG
print('----------------\nClassification 2\n----------------')
classify(train_set, test_set, train_labels, test_labels, 'HOG', 'Gabor', sigma, tau, k, s, 20)
print()

# Detector: Harris,  Descriptor: HOG/HOF
print('----------------\nClassification 5\n----------------')
classify(train_set, test_set, train_labels, test_labels, 'HOG/HOF', 'Harris', sigma, tau, k, s, 20)
print()

# Detector: Gabor,   Descriptor: HOG/HOF
print('----------------\nClassification 6\n----------------')
classify(train_set, test_set, train_labels, test_labels, 'HOG/HOF', 'Gabor', sigma, tau, k, s, 20)
print()

----------------
Classification 2
----------------
Performing Classification using the following parameters:
Detector: Gabor, Descriptor: HOG, sigma = 4.00, tau = 1.50


Classification finished. The results are:
	 Accuracy = 87.88%
	 Pred = [2 2 2 2 2 2 2 2 2 2 2 0 2 0 0 1 0 0 0 2 0 1 1 1 1 1 1 1 1 1 1 1 1]

----------------
Classification 5
----------------
Performing Classification using the following parameters:
Detector: Harris, Descriptor: HOG/HOF, sigma = 4.00, tau = 1.50

✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓
Classification finished. The results are:
	 Accuracy = 78.79%
	 Pred = [0 2 2 0 2 0 2 0 2 0 2 2 0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 1 1 1 1]

----------------
Classification 6
----------------
Performing Classification using the following parameters:
Detector: Gabor, Descriptor: HOG/HOF, sigma = 4.00, tau = 1.50

✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓
Classification finished. The results are:
	 Accuracy = 87.88%
	 Pred = [2 2 2 2 2 2 2 2 2 2 2 0 0 0 0 1 0

In [48]:
# too big training set, very small test set

train_set, test_set, train_labels, test_labels = divide_into_train_test('big_train.txt')
sigma = typ['sigma']
tau = typ['tau']
k = typ['k']
s = typ['s']
num_centers = 20

# Detector: Gabor,   Descriptor: HOG
print('----------------\nClassification 2\n----------------')
classify(train_set, test_set, train_labels, test_labels, 'HOG', 'Gabor', sigma, tau, k, s, 20)
print()

# Detector: Harris,  Descriptor: HOG/HOF
print('----------------\nClassification 5\n----------------')
classify(train_set, test_set, train_labels, test_labels, 'HOG/HOF', 'Harris', sigma, tau, k, s, 20)
print()

# Detector: Gabor,   Descriptor: HOG/HOF
print('----------------\nClassification 6\n----------------')
classify(train_set, test_set, train_labels, test_labels, 'HOG/HOF', 'Gabor', sigma, tau, k, s, 20)
print()

----------------
Classification 2
----------------
Performing Classification using the following parameters:
Detector: Gabor, Descriptor: HOG, sigma = 4.00, tau = 1.50


Classification finished. The results are:
	 Accuracy = 100.00%
	 Pred = [2 2 0 0 1 1]

----------------
Classification 5
----------------
Performing Classification using the following parameters:
Detector: Harris, Descriptor: HOG/HOF, sigma = 4.00, tau = 1.50

✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓
Classification finished. The results are:
	 Accuracy = 66.67%
	 Pred = [2 2 2 0 1 0]

----------------
Classification 6
----------------
Performing Classification using the following parameters:
Detector: Gabor, Descriptor: HOG/HOF, sigma = 4.00, tau = 1.50

✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓
Classification finished. The results are:
	 Accuracy = 100.00%
	 Pred = [2 2 0 0 1 1]



In [49]:
# sufficient train data, but not many box videos

train_set, test_set, train_labels, test_labels = divide_into_train_test('little_box.txt')
sigma = typ['sigma']
tau = typ['tau']
k = typ['k']
s = typ['s']
num_centers = 20

# Detector: Gabor,   Descriptor: HOG
print('----------------\nClassification 2\n----------------')
classify(train_set, test_set, train_labels, test_labels, 'HOG', 'Gabor', sigma, tau, k, s, 20)
print()

# Detector: Harris,  Descriptor: HOG/HOF
print('----------------\nClassification 5\n----------------')
classify(train_set, test_set, train_labels, test_labels, 'HOG/HOF', 'Harris', sigma, tau, k, s, 20)
print()

# Detector: Gabor,   Descriptor: HOG/HOF
print('----------------\nClassification 6\n----------------')
classify(train_set, test_set, train_labels, test_labels, 'HOG/HOF', 'Gabor', sigma, tau, k, s, 20)
print()

----------------
Classification 2
----------------
Performing Classification using the following parameters:
Detector: Gabor, Descriptor: HOG, sigma = 4.00, tau = 1.50


Classification finished. The results are:
	 Accuracy = 50.00%
	 Pred = [2 2 2 2 0 0 1 1 1 2 0 0 1 0 0 0 1 0]

----------------
Classification 5
----------------
Performing Classification using the following parameters:
Detector: Harris, Descriptor: HOG/HOF, sigma = 4.00, tau = 1.50

✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓
Classification finished. The results are:
	 Accuracy = 72.22%
	 Pred = [2 2 0 2 1 1 2 1 0 0 1 1 1 1 1 2 1 1]

----------------
Classification 6
----------------
Performing Classification using the following parameters:
Detector: Gabor, Descriptor: HOG/HOF, sigma = 4.00, tau = 1.50

✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓✓
Classification finished. The results are:
	 Accuracy = 50.00%
	 Pred = [0 2 2 2 0 0 0 1 0 1 0 1 1 0 1 0 0 1]

